# The notebook contains
### Code for _Median_ aggregation algorithm
### Evaluation of all of the attacks (Fang, LIE, and our SOTA AGR-tailored and AGR-agnstic) on Median

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [3]:
import json
from __future__ import print_function
import argparse, os, sys, csv, shutil, time, random, operator, pickle, ast, math
import numpy as np
import pandas as pd
from torch.optim import Optimizer
import torch.nn.functional as F
import torch
import pickle
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data as data
import torch.multiprocessing as mp

sys.path.insert(0,'/home/vshejwalkar/fed-quant-robustness/code/utils/')
from logger import *
from eval import *
from misc import *

from femnist_normal_train import *
from femnist_util import *
from adam import Adam
from sgd import SGD
import torchvision.transforms as transforms
import torchvision.datasets as datasets

## Get the FEMNIST dataset; we use [LEAF framework](https://leaf.cmu.edu/)

In [6]:
user_tr_data = []
user_tr_labels = []

for i in range(34):
    f = '/mnt/nfs/work1/amir/vshejwalkar/leaf/data/femnist/data/train/all_data_%d_niid_0_keep_0_train_9.json'%i
    with open(f, 'r') as myfile:
        data=myfile.read()
    obj = json.loads(data)
    
    for user in obj['users']:
        user_tr_data.append(obj['user_data'][user]['x'])
        user_tr_labels.append(obj['user_data'][user]['y'])

user_te_data = []
user_te_labels = []

for i in range(34):
    f = '/mnt/nfs/work1/amir/vshejwalkar/leaf/data/femnist/data/test/all_data_%d_niid_0_keep_0_test_9.json'%i
    with open(f, 'r') as myfile:
        data=myfile.read()
    obj = json.loads(data)
    
    for user in obj['users']:
        user_te_data.append(obj['user_data'][user]['x'])
        user_te_labels.append(obj['user_data'][user]['y'])

In [7]:
user_tr_data_tensors=[]
user_tr_label_tensors=[]

for i in range(len(user_tr_data)):
    
    user_tr_data_tensor=torch.from_numpy(np.array(user_tr_data[i])).type(torch.FloatTensor)
    user_tr_label_tensor=torch.from_numpy(np.array(user_tr_labels[i])).type(torch.LongTensor)

    user_tr_data_tensors.append(user_tr_data_tensor)
    user_tr_label_tensors.append(user_tr_label_tensor)
    
    print('user %d tr len %d'%(i,len(user_tr_data_tensor)))

user 0 tr len 333
user 1 tr len 310
user 2 tr len 375
user 3 tr len 344
user 4 tr len 319
user 5 tr len 218
user 6 tr len 119
user 7 tr len 275
user 8 tr len 270
user 9 tr len 377
user 10 tr len 335
user 11 tr len 346
user 12 tr len 327
user 13 tr len 358
user 14 tr len 337
user 15 tr len 354
user 16 tr len 369
user 17 tr len 367
user 18 tr len 276
user 19 tr len 299
user 20 tr len 348
user 21 tr len 342
user 22 tr len 334
user 23 tr len 378
user 24 tr len 333
user 25 tr len 390
user 26 tr len 160
user 27 tr len 351
user 28 tr len 363
user 29 tr len 378
user 30 tr len 337
user 31 tr len 354
user 32 tr len 267
user 33 tr len 398
user 34 tr len 289
user 35 tr len 351
user 36 tr len 387
user 37 tr len 390
user 38 tr len 225
user 39 tr len 344
user 40 tr len 323
user 41 tr len 377
user 42 tr len 292
user 43 tr len 324
user 44 tr len 185
user 45 tr len 376
user 46 tr len 225
user 47 tr len 373
user 48 tr len 360
user 49 tr len 385
user 50 tr len 324
user 51 tr len 321
user 52 tr len 353
use

user 416 tr len 315
user 417 tr len 356
user 418 tr len 336
user 419 tr len 322
user 420 tr len 329
user 421 tr len 379
user 422 tr len 285
user 423 tr len 271
user 424 tr len 293
user 425 tr len 281
user 426 tr len 358
user 427 tr len 282
user 428 tr len 324
user 429 tr len 333
user 430 tr len 141
user 431 tr len 338
user 432 tr len 276
user 433 tr len 345
user 434 tr len 248
user 435 tr len 314
user 436 tr len 204
user 437 tr len 293
user 438 tr len 261
user 439 tr len 372
user 440 tr len 194
user 441 tr len 344
user 442 tr len 312
user 443 tr len 274
user 444 tr len 199
user 445 tr len 367
user 446 tr len 368
user 447 tr len 209
user 448 tr len 342
user 449 tr len 271
user 450 tr len 271
user 451 tr len 336
user 452 tr len 297
user 453 tr len 321
user 454 tr len 393
user 455 tr len 314
user 456 tr len 238
user 457 tr len 287
user 458 tr len 374
user 459 tr len 234
user 460 tr len 313
user 461 tr len 217
user 462 tr len 301
user 463 tr len 353
user 464 tr len 355
user 465 tr len 362


user 840 tr len 145
user 841 tr len 152
user 842 tr len 153
user 843 tr len 164
user 844 tr len 152
user 845 tr len 126
user 846 tr len 163
user 847 tr len 135
user 848 tr len 159
user 849 tr len 140
user 850 tr len 160
user 851 tr len 158
user 852 tr len 155
user 853 tr len 156
user 854 tr len 160
user 855 tr len 152
user 856 tr len 152
user 857 tr len 143
user 858 tr len 155
user 859 tr len 142
user 860 tr len 160
user 861 tr len 155
user 862 tr len 157
user 863 tr len 157
user 864 tr len 158
user 865 tr len 154
user 866 tr len 162
user 867 tr len 122
user 868 tr len 152
user 869 tr len 150
user 870 tr len 157
user 871 tr len 161
user 872 tr len 164
user 873 tr len 135
user 874 tr len 148
user 875 tr len 142
user 876 tr len 158
user 877 tr len 135
user 878 tr len 156
user 879 tr len 161
user 880 tr len 151
user 881 tr len 151
user 882 tr len 154
user 883 tr len 164
user 884 tr len 153
user 885 tr len 164
user 886 tr len 136
user 887 tr len 152
user 888 tr len 146
user 889 tr len 153


user 1264 tr len 122
user 1265 tr len 147
user 1266 tr len 138
user 1267 tr len 154
user 1268 tr len 149
user 1269 tr len 90
user 1270 tr len 127
user 1271 tr len 111
user 1272 tr len 155
user 1273 tr len 161
user 1274 tr len 158
user 1275 tr len 88
user 1276 tr len 138
user 1277 tr len 151
user 1278 tr len 78
user 1279 tr len 147
user 1280 tr len 94
user 1281 tr len 135
user 1282 tr len 145
user 1283 tr len 143
user 1284 tr len 149
user 1285 tr len 37
user 1286 tr len 156
user 1287 tr len 151
user 1288 tr len 144
user 1289 tr len 97
user 1290 tr len 123
user 1291 tr len 144
user 1292 tr len 148
user 1293 tr len 161
user 1294 tr len 101
user 1295 tr len 144
user 1296 tr len 153
user 1297 tr len 141
user 1298 tr len 153
user 1299 tr len 143
user 1300 tr len 149
user 1301 tr len 135
user 1302 tr len 117
user 1303 tr len 142
user 1304 tr len 146
user 1305 tr len 161
user 1306 tr len 155
user 1307 tr len 121
user 1308 tr len 138
user 1309 tr len 151
user 1310 tr len 138
user 1311 tr len 12

user 1660 tr len 157
user 1661 tr len 154
user 1662 tr len 155
user 1663 tr len 158
user 1664 tr len 162
user 1665 tr len 160
user 1666 tr len 129
user 1667 tr len 148
user 1668 tr len 153
user 1669 tr len 162
user 1670 tr len 162
user 1671 tr len 145
user 1672 tr len 157
user 1673 tr len 142
user 1674 tr len 154
user 1675 tr len 141
user 1676 tr len 150
user 1677 tr len 153
user 1678 tr len 149
user 1679 tr len 145
user 1680 tr len 153
user 1681 tr len 155
user 1682 tr len 157
user 1683 tr len 156
user 1684 tr len 142
user 1685 tr len 161
user 1686 tr len 155
user 1687 tr len 134
user 1688 tr len 157
user 1689 tr len 164
user 1690 tr len 152
user 1691 tr len 148
user 1692 tr len 147
user 1693 tr len 160
user 1694 tr len 158
user 1695 tr len 153
user 1696 tr len 159
user 1697 tr len 157
user 1698 tr len 161
user 1699 tr len 161
user 1700 tr len 156
user 1701 tr len 153
user 1702 tr len 143
user 1703 tr len 148
user 1704 tr len 153
user 1705 tr len 160
user 1706 tr len 163
user 1707 tr 

user 2055 tr len 153
user 2056 tr len 153
user 2057 tr len 143
user 2058 tr len 163
user 2059 tr len 161
user 2060 tr len 128
user 2061 tr len 158
user 2062 tr len 126
user 2063 tr len 145
user 2064 tr len 124
user 2065 tr len 147
user 2066 tr len 135
user 2067 tr len 162
user 2068 tr len 126
user 2069 tr len 147
user 2070 tr len 162
user 2071 tr len 148
user 2072 tr len 147
user 2073 tr len 153
user 2074 tr len 153
user 2075 tr len 140
user 2076 tr len 150
user 2077 tr len 142
user 2078 tr len 162
user 2079 tr len 127
user 2080 tr len 160
user 2081 tr len 164
user 2082 tr len 151
user 2083 tr len 158
user 2084 tr len 152
user 2085 tr len 147
user 2086 tr len 134
user 2087 tr len 148
user 2088 tr len 131
user 2089 tr len 153
user 2090 tr len 152
user 2091 tr len 145
user 2092 tr len 158
user 2093 tr len 153
user 2094 tr len 161
user 2095 tr len 159
user 2096 tr len 158
user 2097 tr len 141
user 2098 tr len 127
user 2099 tr len 161
user 2100 tr len 162
user 2101 tr len 147
user 2102 tr 

user 2454 tr len 161
user 2455 tr len 162
user 2456 tr len 152
user 2457 tr len 138
user 2458 tr len 153
user 2459 tr len 158
user 2460 tr len 159
user 2461 tr len 152
user 2462 tr len 159
user 2463 tr len 138
user 2464 tr len 163
user 2465 tr len 158
user 2466 tr len 127
user 2467 tr len 148
user 2468 tr len 146
user 2469 tr len 160
user 2470 tr len 162
user 2471 tr len 148
user 2472 tr len 153
user 2473 tr len 162
user 2474 tr len 164
user 2475 tr len 164
user 2476 tr len 161
user 2477 tr len 163
user 2478 tr len 140
user 2479 tr len 166
user 2480 tr len 164
user 2481 tr len 153
user 2482 tr len 144
user 2483 tr len 164
user 2484 tr len 143
user 2485 tr len 136
user 2486 tr len 165
user 2487 tr len 165
user 2488 tr len 137
user 2489 tr len 160
user 2490 tr len 146
user 2491 tr len 153
user 2492 tr len 147
user 2493 tr len 154
user 2494 tr len 156
user 2495 tr len 155
user 2496 tr len 162
user 2497 tr len 142
user 2498 tr len 154
user 2499 tr len 158
user 2500 tr len 151
user 2501 tr 

user 2850 tr len 327
user 2851 tr len 296
user 2852 tr len 305
user 2853 tr len 221
user 2854 tr len 198
user 2855 tr len 232
user 2856 tr len 255
user 2857 tr len 341
user 2858 tr len 358
user 2859 tr len 288
user 2860 tr len 275
user 2861 tr len 325
user 2862 tr len 356
user 2863 tr len 331
user 2864 tr len 216
user 2865 tr len 362
user 2866 tr len 338
user 2867 tr len 352
user 2868 tr len 319
user 2869 tr len 335
user 2870 tr len 270
user 2871 tr len 298
user 2872 tr len 297
user 2873 tr len 260
user 2874 tr len 312
user 2875 tr len 360
user 2876 tr len 215
user 2877 tr len 284
user 2878 tr len 254
user 2879 tr len 277
user 2880 tr len 354
user 2881 tr len 289
user 2882 tr len 285
user 2883 tr len 333
user 2884 tr len 352
user 2885 tr len 231
user 2886 tr len 226
user 2887 tr len 298
user 2888 tr len 375
user 2889 tr len 353
user 2890 tr len 337
user 2891 tr len 288
user 2892 tr len 354
user 2893 tr len 363
user 2894 tr len 303
user 2895 tr len 264
user 2896 tr len 351
user 2897 tr 

user 3257 tr len 317
user 3258 tr len 309
user 3259 tr len 240
user 3260 tr len 284
user 3261 tr len 213
user 3262 tr len 152
user 3263 tr len 192
user 3264 tr len 162
user 3265 tr len 235
user 3266 tr len 200
user 3267 tr len 312
user 3268 tr len 242
user 3269 tr len 273
user 3270 tr len 192
user 3271 tr len 230
user 3272 tr len 260
user 3273 tr len 221
user 3274 tr len 270
user 3275 tr len 186
user 3276 tr len 297
user 3277 tr len 134
user 3278 tr len 198
user 3279 tr len 199
user 3280 tr len 243
user 3281 tr len 342
user 3282 tr len 153
user 3283 tr len 184
user 3284 tr len 209
user 3285 tr len 252
user 3286 tr len 283
user 3287 tr len 233
user 3288 tr len 235
user 3289 tr len 162
user 3290 tr len 297
user 3291 tr len 257
user 3292 tr len 208
user 3293 tr len 252
user 3294 tr len 229
user 3295 tr len 322
user 3296 tr len 254
user 3297 tr len 209
user 3298 tr len 242
user 3299 tr len 328
user 3300 tr len 296
user 3301 tr len 339
user 3302 tr len 294
user 3303 tr len 224
user 3304 tr 

In [8]:
te_data = np.concatenate(user_te_data, 0)
te_labels = np.concatenate(user_te_labels)
te_len = len(te_labels)

te_data_tensor = torch.from_numpy(te_data[:(te_len//2)]).type(torch.FloatTensor)
te_label_tensor = torch.from_numpy(te_labels[:(te_len//2)]).type(torch.LongTensor)

val_data_tensor = torch.from_numpy(te_data[(te_len//2):]).type(torch.FloatTensor)
val_label_tensor = torch.from_numpy(te_labels[(te_len//2):]).type(torch.LongTensor)

## Model architecture for FEMNIST

In [9]:
class mnist_conv(nn.Module):
    def __init__(self):
        super(mnist_conv, self).__init__()

        self.conv1 = nn.Conv2d(1, 16, 5, padding=2)
        self.conv2 = nn.Conv2d(16, 32, 5, padding=2)
        self.fc1 = nn.Linear(32 * 7 * 7, 512)
        self.fc2 = nn.Linear(512, 62)

    def forward(self, x, noise=torch.Tensor()):
        x = x.reshape(-1, 1, 28, 28)

        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, 32 * 7 * 7)  # reshape Variable
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        torch.nn.init.xavier_uniform_(m.weight)
    elif classname.find('Linear') != -1:
        torch.nn.init.xavier_uniform_(m.weight)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.fill_(0)
        m.bias.data.fill_(0)

## Code for (Full-knowledge) Fang attack on Median

### Note that the Fang attacks on Trimmed-mean and Median are the same 

In [10]:
def get_malicious_updates_fang_trmean(all_updates, deviation, n_attackers, epoch_num):
    b = 2
    max_vector = torch.max(all_updates, 0)[0]
    min_vector = torch.min(all_updates, 0)[0]

    max_ = (max_vector > 0).type(torch.FloatTensor).cuda()
    min_ = (min_vector < 0).type(torch.FloatTensor).cuda()

    max_[max_ == 1] = b
    max_[max_ == 0] = 1 / b
    min_[min_ == 1] = b
    min_[min_ == 0] = 1 / b

    max_range = torch.cat((max_vector[:, None], (max_vector * max_)[:, None]), dim=1)
    min_range = torch.cat(((min_vector * min_)[:, None], min_vector[:, None]), dim=1)

    rand = torch.from_numpy(np.random.uniform(0, 1, [len(deviation), n_attackers])).type(torch.FloatTensor).cuda()

    max_rand = torch.stack([max_range[:, 0]] * rand.shape[1]).T + rand * torch.stack([max_range[:, 1] - max_range[:, 0]] * rand.shape[1]).T
    min_rand = torch.stack([min_range[:, 0]] * rand.shape[1]).T + rand * torch.stack([min_range[:, 1] - min_range[:, 0]] * rand.shape[1]).T

    mal_vec = (torch.stack([(deviation > 0).type(torch.FloatTensor)] * max_rand.shape[1]).T.cuda() * max_rand + torch.stack(
        [(deviation > 0).type(torch.FloatTensor)] * min_rand.shape[1]).T.cuda() * min_rand).T

    mal_updates = torch.cat((mal_vec, all_updates), 0)

    return mal_updates

## Evaluation for Full-knolwledge Fang attack on Median

In [14]:
resume=0
nepochs=1500
gamma=.1
fed_lr=0.001

criterion = nn.CrossEntropyLoss()
use_cuda = torch.cuda.is_available()
batch_size = 100
schedule = [5000]

aggregation = 'median'
chkpt = './' + aggregation

at_type='fang'
at_fractions = [20]

for at_fraction in at_fractions:
    epoch_num = 0

    fed_model = mnist_conv().cuda()
    fed_model.apply(weights_init)
    optimizer_fed = Adam(fed_model.parameters(), lr=fed_lr)

    best_global_acc=0
    best_global_te_acc=0

    while epoch_num <= nepochs:
        user_grads = []

        round_users = np.random.choice(3400, 60)
        n_attacker = np.sum(round_users < (34*at_fraction))

        for i in round_users:
            if i < (34*at_fraction):
                continue

            inputs = user_tr_data_tensors[i]
            targets = user_tr_label_tensors[i]

            inputs, targets = inputs.cuda(), targets.cuda()
            inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

            outputs = fed_model(inputs)
            loss = criterion(outputs, targets)
            optimizer_fed.zero_grad()
            loss.backward(retain_graph=True)

            param_grad=[]
            for param in fed_model.parameters():
                param_grad=param.grad.data.view(-1) if not len(param_grad) else torch.cat((param_grad,param.grad.view(-1)))

            user_grads=param_grad[None,:] if len(user_grads)==0 else torch.cat((user_grads,param_grad[None,:]),0)    

        malicious_grads = user_grads

        if n_attacker:
            if at_type == 'fang':
                agg_grads = torch.mean(malicious_grads, 0)
                deviation = torch.sign(agg_grads)
                malicious_grads = get_malicious_updates_fang_trmean(malicious_grads, deviation, n_attacker, epoch_num)
            elif at_type == 'our-agr':
                pass
        
        if epoch_num == 0: print('malicious grads shape ', malicious_grads.shape)
        
        if aggregation=='median':
            agg_grads=torch.median(malicious_grads,dim=0)[0]
        elif aggregation=='average':
            agg_grads=torch.mean(malicious_grads, dim=0)
        elif aggregation=='trmean':
            agg_grads=tr_mean(malicious_grads, n_attacker)

        start_idx=0

        if epoch_num in schedule:
            for param_group in optimizer_fed.param_groups:
                param_group['lr'] *= gamma
                print('New learnin rate ', param_group['lr'])

        optimizer_fed.zero_grad()

        model_grads=[]

        for i, param in enumerate(fed_model.parameters()):
            param_=agg_grads[start_idx:start_idx+len(param.data.view(-1))].reshape(param.data.shape)
            start_idx=start_idx+len(param.data.view(-1))
            param_=param_.cuda()
            model_grads.append(param_)

        optimizer_fed.step(model_grads)

        val_loss, val_acc = test(val_data_tensor,val_label_tensor,fed_model,criterion,use_cuda)
        te_loss, te_acc = test(te_data_tensor,te_label_tensor, fed_model, criterion, use_cuda)

        is_best = best_global_acc < val_acc

        best_global_acc = max(best_global_acc, val_acc)

        if is_best:
            best_global_te_acc = te_acc

        if epoch_num % 10 == 0 or epoch_num == nepochs-1:
            print('%s: at %s n_at %d e %d | val loss %.4f val acc %.4f best val_acc %f te_acc %f'%(aggregation, at_type, n_attacker, epoch_num, val_loss, val_acc, best_global_acc,best_global_te_acc))

        epoch_num+=1

malicious grads shape  torch.Size([60, 848382])


/home/vshejwalkar/fed-quant-robustness/code/femnist/adam.py:76: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729138878/work/torch/csrc/utils/python_arg_parser.cpp:882.)
  exp_avg.mul_(beta1).add_(1 - beta1, grad)


median: at fang n_at 11 e 0 | val loss 3.9690 val acc 5.3336 best val_acc 5.333608 te_acc 5.781507
median: at fang n_at 11 e 10 | val loss 3.7899 val acc 4.8214 best val_acc 7.825371 te_acc 8.641371
median: at fang n_at 11 e 20 | val loss 3.7719 val acc 6.8343 best val_acc 15.957063 te_acc 17.437191
median: at fang n_at 12 e 30 | val loss 3.7286 val acc 19.2777 best val_acc 19.277698 te_acc 21.481157
median: at fang n_at 16 e 40 | val loss 3.6217 val acc 23.7412 best val_acc 24.595861 te_acc 27.393946
median: at fang n_at 13 e 50 | val loss 3.4656 val acc 29.2293 best val_acc 30.276977 te_acc 33.976009
median: at fang n_at 10 e 60 | val loss 3.2946 val acc 31.5357 best val_acc 31.546026 te_acc 35.278521
median: at fang n_at 12 e 70 | val loss 3.0165 val acc 32.7662 best val_acc 33.283567 te_acc 37.587521
median: at fang n_at 14 e 80 | val loss 2.7453 val acc 35.2682 best val_acc 35.268225 te_acc 39.613365
median: at fang n_at 7 e 90 | val loss 2.5632 val acc 36.7612 best val_acc 36.761

median: at fang n_at 17 e 800 | val loss 0.9707 val acc 70.7707 best val_acc 72.438736 te_acc 73.890548
median: at fang n_at 8 e 810 | val loss 0.9127 val acc 71.8647 best val_acc 72.438736 te_acc 73.890548
median: at fang n_at 12 e 820 | val loss 0.9100 val acc 71.4631 best val_acc 72.438736 te_acc 73.890548
median: at fang n_at 11 e 830 | val loss 0.8941 val acc 72.1762 best val_acc 72.467051 te_acc 73.908567
median: at fang n_at 17 e 840 | val loss 0.9017 val acc 72.4979 best val_acc 72.778521 te_acc 74.209741
median: at fang n_at 8 e 850 | val loss 0.9027 val acc 72.1015 best val_acc 72.778521 te_acc 74.209741
median: at fang n_at 10 e 860 | val loss 0.8817 val acc 72.1710 best val_acc 72.799114 te_acc 74.145387
median: at fang n_at 12 e 870 | val loss 0.8894 val acc 72.0372 best val_acc 73.646005 te_acc 74.724568
median: at fang n_at 9 e 880 | val loss 0.8986 val acc 71.8801 best val_acc 73.646005 te_acc 74.724568
median: at fang n_at 10 e 890 | val loss 0.8847 val acc 71.9780 bes

## Code for our SOTA AGR-tailored attack on Median

In [15]:
def our_attack_median(all_updates, n_attackers, dev_type='sign', threshold=5.0, threshold_diff=1e-5):
    
    model_re = torch.mean(all_updates, 0)
    
    if dev_type == 'sign':
        deviation = torch.sign(model_re)
    elif dev_type == 'unit_vec':
        deviation = model_re / torch.norm(model_re)  # unit vector, dir opp to good dir
    elif dev_type == 'std':
        deviation = torch.std(all_updates, 0)

    lamda = torch.Tensor([threshold]).cuda()  # compute_lambda_our(all_updates, model_re, n_attackers)

    threshold_diff = threshold_diff
    prev_loss = -1
    lamda_fail = lamda
    lamda_succ = 0

    while torch.abs(lamda_succ - lamda) > threshold_diff:
        mal_update = (model_re - lamda * deviation)
        mal_updates = torch.stack([mal_update] * n_attackers)
        mal_updates = torch.cat((mal_updates, all_updates), 0)

        agg_grads = torch.median(mal_updates, 0)[0]

        loss = torch.norm(agg_grads - model_re)

        if prev_loss < loss:
            # print('successful lamda is ', lamda)
            lamda_succ = lamda
            lamda = lamda + lamda_fail / 2
        else:
            lamda = lamda - lamda_fail / 2

        lamda_fail = lamda_fail / 2
        prev_loss = loss

    mal_update = (model_re - lamda_succ * deviation)
    
    return mal_update

## Evaluation of our SOTA AGR-tailored attack on Median

In [18]:
resume=0
nepochs=1500
gamma=.1
fed_lr=0.001

criterion = nn.CrossEntropyLoss()
use_cuda = torch.cuda.is_available()
batch_size = 100
schedule = [5000]

aggregation = 'median'
chkpt = './' + aggregation

at_type='our-agr'
at_fractions = [20]

for at_fraction in at_fractions:
    epoch_num = 0

    fed_model = mnist_conv().cuda()
    fed_model.apply(weights_init)
    optimizer_fed = Adam(fed_model.parameters(), lr=fed_lr)

    best_global_acc=0
    best_global_te_acc=0

    while epoch_num <= nepochs:
        user_grads = []

        round_users = np.random.choice(3400, 60)
        n_attacker = np.sum(round_users < (34*at_fraction))

        for i in round_users:
            if i < (34*at_fraction):
                continue

            inputs = user_tr_data_tensors[i]
            targets = user_tr_label_tensors[i]

            inputs, targets = inputs.cuda(), targets.cuda()
            inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

            outputs = fed_model(inputs)
            loss = criterion(outputs, targets)
            optimizer_fed.zero_grad()
            loss.backward(retain_graph=True)

            param_grad=[]
            for param in fed_model.parameters():
                param_grad=param.grad.data.view(-1) if not len(param_grad) else torch.cat((param_grad,param.grad.view(-1)))

            user_grads=param_grad[None,:] if len(user_grads)==0 else torch.cat((user_grads,param_grad[None,:]),0)    

        malicious_grads = user_grads

        if n_attacker:
            if at_type == 'fang':
                agg_grads = torch.mean(malicious_grads, 0)
                deviation = torch.sign(agg_grads)
                malicious_grads = get_malicious_updates_fang_trmean(malicious_grads, deviation, n_attacker, epoch_num)
            elif at_type == 'our-agr':
                mal_update = our_attack_median(malicious_grads, n_attacker, dev_type='sign', threshold=5.0, threshold_diff=1e-5)
                malicious_grads = torch.cat((torch.stack([mal_update]*n_attacker), malicious_grads))
                
        if malicious_grads.shape[0] != 60 or not epoch_num: print('malicious grads shape ', malicious_grads.shape)
        
        if aggregation=='median':
            agg_grads=torch.median(malicious_grads,dim=0)[0]
        elif aggregation=='average':
            agg_grads=torch.mean(malicious_grads, dim=0)
        elif aggregation=='trmean':
            agg_grads=tr_mean(malicious_grads, n_attacker)

        start_idx=0

        if epoch_num in schedule:
            for param_group in optimizer_fed.param_groups:
                param_group['lr'] *= gamma
                print('New learnin rate ', param_group['lr'])

        optimizer_fed.zero_grad()

        model_grads=[]

        for i, param in enumerate(fed_model.parameters()):
            param_=agg_grads[start_idx:start_idx+len(param.data.view(-1))].reshape(param.data.shape)
            start_idx=start_idx+len(param.data.view(-1))
            param_=param_.cuda()
            model_grads.append(param_)

        optimizer_fed.step(model_grads)

        val_loss, val_acc = test(val_data_tensor,val_label_tensor,fed_model,criterion,use_cuda)
        te_loss, te_acc = test(te_data_tensor,te_label_tensor, fed_model, criterion, use_cuda)

        is_best = best_global_acc < val_acc

        best_global_acc = max(best_global_acc, val_acc)

        if is_best:
            best_global_te_acc = te_acc

        if epoch_num % 10 == 0 or epoch_num == nepochs-1:
            print('%s: at %s n_at %d e %d | val loss %.4f val acc %.4f best val_acc %f te_acc %f'%(aggregation, at_type, n_attacker, epoch_num, val_loss, val_acc, best_global_acc,best_global_te_acc))

        epoch_num+=1

malicious grads shape  torch.Size([60, 848382])
median: at our-agr n_at 9 e 0 | val loss 3.9986 val acc 4.8986 best val_acc 4.898579 te_acc 5.403110
median: at our-agr n_at 10 e 10 | val loss 3.7912 val acc 12.7471 best val_acc 12.747117 te_acc 13.789642
median: at our-agr n_at 10 e 20 | val loss 3.8561 val acc 15.3161 best val_acc 16.441001 te_acc 18.139930
median: at our-agr n_at 14 e 30 | val loss 3.8118 val acc 20.5416 best val_acc 26.266474 te_acc 29.458402
median: at our-agr n_at 7 e 40 | val loss 3.6176 val acc 28.5832 best val_acc 31.000309 te_acc 34.416186
median: at our-agr n_at 14 e 50 | val loss 3.3989 val acc 32.7327 best val_acc 32.732702 te_acc 36.655684
median: at our-agr n_at 15 e 60 | val loss 3.3995 val acc 29.2962 best val_acc 32.732702 te_acc 36.655684
median: at our-agr n_at 7 e 70 | val loss 3.0007 val acc 30.8150 best val_acc 33.100803 te_acc 36.985173
median: at our-agr n_at 6 e 80 | val loss 2.7274 val acc 32.9644 best val_acc 34.557764 te_acc 38.560544
median

median: at our-agr n_at 13 e 770 | val loss 3.8688 val acc 34.9645 best val_acc 45.279036 te_acc 47.842875
median: at our-agr n_at 15 e 780 | val loss 3.8979 val acc 36.5167 best val_acc 45.279036 te_acc 47.842875
median: at our-agr n_at 17 e 790 | val loss 3.8346 val acc 36.9672 best val_acc 45.279036 te_acc 47.842875
median: at our-agr n_at 6 e 800 | val loss 4.1446 val acc 37.7986 best val_acc 45.279036 te_acc 47.842875
median: at our-agr n_at 10 e 810 | val loss 4.2521 val acc 38.2722 best val_acc 45.279036 te_acc 47.842875
median: at our-agr n_at 12 e 820 | val loss 4.0767 val acc 38.0689 best val_acc 45.279036 te_acc 47.842875
median: at our-agr n_at 12 e 830 | val loss 4.5735 val acc 35.0546 best val_acc 45.279036 te_acc 47.842875
median: at our-agr n_at 22 e 840 | val loss 5.0179 val acc 32.7224 best val_acc 45.279036 te_acc 47.842875
median: at our-agr n_at 19 e 850 | val loss 4.2900 val acc 37.0959 best val_acc 45.279036 te_acc 47.842875
median: at our-agr n_at 14 e 860 | val

## Code for our first SOTA AGR-agnostic attack - Min-max

In [19]:
def our_attack_dist(all_updates, model_re, n_attackers, dev_type='unit_vec'):

    if dev_type == 'unit_vec':
        deviation = model_re / torch.norm(model_re)  # unit vector, dir opp to good dir
    elif dev_type == 'sign':
        deviation = torch.sign(model_re)
    elif dev_type == 'std':
        deviation = torch.std(all_updates, 0)

    lamda = torch.Tensor([50.0]).float().cuda()
    # print(lamda)
    threshold_diff = 1e-5
    lamda_fail = lamda
    lamda_succ = 0
    
    distances = []
    for update in all_updates:
        distance = torch.norm((all_updates - update), dim=1) ** 2
        distances = distance[None, :] if not len(distances) else torch.cat((distances, distance[None, :]), 0)
    
    max_distance = torch.max(distances)
    del distances

    while torch.abs(lamda_succ - lamda) > threshold_diff:
        mal_update = (model_re - lamda * deviation)
        distance = torch.norm((all_updates - mal_update), dim=1) ** 2
        max_d = torch.max(distance)
        
        if max_d <= max_distance:
            # print('successful lamda is ', lamda)
            lamda_succ = lamda
            lamda = lamda + lamda_fail / 2
        else:
            lamda = lamda - lamda_fail / 2

        lamda_fail = lamda_fail / 2

    mal_update = (model_re - lamda_succ * deviation)
    
    return mal_update

In [20]:
resume=0
nepochs=1500
gamma=.1
fed_lr=0.001

criterion = nn.CrossEntropyLoss()
use_cuda = torch.cuda.is_available()
batch_size = 100
schedule = [10000]
gamma=.5

aggregation='median'
candidates = []

at_type='min-max'
dev_type='sign'
at_fractions=[20]

chkpt='./'+aggregation


for at_fraction in at_fractions:
    epoch_num = 0
    best_global_acc = 0
    best_global_te_acc = 0

    print('\n====> dev type %s batch size %d lr %f'% (dev_type, batch_size, fed_lr))    

    torch.cuda.empty_cache()

    fed_model = mnist_conv().cuda()
    fed_model.apply(weights_init)
    optimizer_fed = Adam(fed_model.parameters(), lr=fed_lr)

    while epoch_num <= nepochs:
        user_grads = []
        round_users = np.random.choice(3400, 60)
        n_attacker = np.sum(round_users < (34*at_fraction))

        for i in round_users:
            if i < (34*at_fraction):
                continue

            inputs = user_tr_data_tensors[i]
            targets = user_tr_label_tensors[i]

            inputs, targets = inputs.cuda(), targets.cuda()
            inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

            outputs = fed_model(inputs)
            loss = criterion(outputs, targets)
            optimizer_fed.zero_grad()
            loss.backward(retain_graph=True)

            param_grad=[]
            for param in fed_model.parameters():
                param_grad=param.grad.data.view(-1) if not len(param_grad) else torch.cat((param_grad,param.grad.view(-1)))

            user_grads=param_grad[None,:] if len(user_grads)==0 else torch.cat((user_grads,param_grad[None,:]),0)    

        malicious_grads = user_grads

        if epoch_num in schedule:
            for param_group in optimizer_fed.param_groups:
                param_group['lr'] *= gamma
                print('New learnin rate ', param_group['lr'])

        if n_attacker > 0:
            if at_type == 'lie':
                malicious_grads = get_malicious_updates_lie(malicious_grads, n_attacker, z, epoch_num)
            elif at_type == 'fang':
                agg_grads = torch.mean(malicious_grads, 0)
                deviation = torch.sign(agg_grads)
                malicious_grads = get_malicious_updates_fang(malicious_grads, agg_grads, deviation, n_attacker)
            elif at_type == 'our_attack_trmean':
                agg_grads = torch.mean(malicious_grads, 0)
                mal_update = our_attack_trmean(malicious_grads, agg_grads, n_attacker, dev_type)
            elif at_type == 'our_attack_median':
                agg_grads = torch.mean(malicious_grads, 0)
                mal_update = our_attack_median(malicious_grads, agg_grads, n_attacker, dev_type)
            elif at_type == 'min-max':
                agg_grads = torch.mean(malicious_grads, 0)
                mal_update = our_attack_dist(malicious_grads, agg_grads, n_attacker, dev_type)
            elif at_type == 'min-sum':
                agg_grads = torch.mean(malicious_grads, 0)
                mal_update = our_attack_score(malicious_grads, agg_grads, n_attacker, dev_type)

            mal_updates = torch.stack([mal_update] * n_attacker)
            malicious_grads = torch.cat((mal_updates, user_grads), 0)

        if epoch_num==0: print('malicious_grads shape ', malicious_grads.shape)

        if aggregation=='median':
            agg_grads=torch.median(malicious_grads,dim=0)[0]

        elif aggregation=='average':
            agg_grads=torch.mean(malicious_grads,dim=0)

        elif aggregation=='trmean':
            agg_grads=tr_mean(malicious_grads, n_attacker)

        del user_grads

        start_idx=0

        optimizer_fed.zero_grad()

        model_grads=[]

        for i, param in enumerate(fed_model.parameters()):
            param_=agg_grads[start_idx:start_idx+len(param.data.view(-1))].reshape(param.data.shape)
            start_idx=start_idx+len(param.data.view(-1))
            param_=param_.cuda()
            model_grads.append(param_)

        optimizer_fed.step(model_grads)

        val_loss, val_acc = test(val_data_tensor,val_label_tensor,fed_model,criterion,use_cuda)
        te_loss, te_acc = test(te_data_tensor,te_label_tensor, fed_model, criterion, use_cuda)

        is_best = best_global_acc < val_acc

        best_global_acc = max(best_global_acc, val_acc)

        if is_best:
            best_global_te_acc = te_acc

        if epoch_num%10==0 or epoch_num==nepochs-1:
            print('SB %s: at %s n_at %d e %d fed_model val loss %.4f val acc %.4f best val_acc %f te_acc %f'%(aggregation, at_type, n_attacker, epoch_num, val_loss, val_acc, best_global_acc,best_global_te_acc))

        epoch_num+=1


====> dev type sign batch size 100 lr 0.001000
malicious_grads shape  torch.Size([60, 848382])
SB median: at min-max n_at 11 e 0 fed_model val loss 3.9489 val acc 4.5923 best val_acc 4.592257 te_acc 5.212624
SB median: at min-max n_at 8 e 10 fed_model val loss 3.8180 val acc 4.7390 best val_acc 11.151153 te_acc 12.090713
SB median: at min-max n_at 13 e 20 fed_model val loss 3.8398 val acc 10.4561 best val_acc 11.923394 te_acc 13.370058
SB median: at min-max n_at 13 e 30 fed_model val loss 3.8113 val acc 27.8213 best val_acc 28.320634 te_acc 31.525432
SB median: at min-max n_at 15 e 40 fed_model val loss 3.6335 val acc 28.5394 best val_acc 29.383752 te_acc 32.544790
SB median: at min-max n_at 14 e 50 fed_model val loss 3.3284 val acc 31.1496 best val_acc 31.149609 te_acc 34.990218
SB median: at min-max n_at 9 e 60 fed_model val loss 3.0769 val acc 33.1986 best val_acc 33.198620 te_acc 36.846170
SB median: at min-max n_at 11 e 70 fed_model val loss 2.9080 val acc 34.8100 best val_acc 34

SB median: at min-max n_at 20 e 690 fed_model val loss 2.8775 val acc 43.1348 best val_acc 45.214683 te_acc 49.248353
SB median: at min-max n_at 6 e 700 fed_model val loss 2.9348 val acc 40.7434 best val_acc 45.214683 te_acc 49.248353
SB median: at min-max n_at 16 e 710 fed_model val loss 2.8686 val acc 43.0884 best val_acc 45.214683 te_acc 49.248353
SB median: at min-max n_at 10 e 720 fed_model val loss 2.7656 val acc 43.3639 best val_acc 45.214683 te_acc 49.248353
SB median: at min-max n_at 12 e 730 fed_model val loss 2.6623 val acc 43.7835 best val_acc 45.214683 te_acc 49.248353
SB median: at min-max n_at 9 e 740 fed_model val loss 2.6840 val acc 44.6046 best val_acc 45.214683 te_acc 49.248353
SB median: at min-max n_at 16 e 750 fed_model val loss 2.6486 val acc 42.6508 best val_acc 45.894255 te_acc 49.178851
SB median: at min-max n_at 11 e 760 fed_model val loss 2.6971 val acc 42.7203 best val_acc 45.894255 te_acc 49.178851
SB median: at min-max n_at 10 e 770 fed_model val loss 2.7

SB median: at min-max n_at 15 e 1390 fed_model val loss 2.3015 val acc 51.8405 best val_acc 51.976936 te_acc 55.524094
SB median: at min-max n_at 17 e 1400 fed_model val loss 2.3922 val acc 49.5624 best val_acc 51.976936 te_acc 55.524094
SB median: at min-max n_at 8 e 1410 fed_model val loss 2.3071 val acc 49.1660 best val_acc 51.976936 te_acc 55.524094
SB median: at min-max n_at 16 e 1420 fed_model val loss 2.3492 val acc 51.4106 best val_acc 51.976936 te_acc 55.524094
SB median: at min-max n_at 14 e 1430 fed_model val loss 2.6991 val acc 43.3124 best val_acc 51.976936 te_acc 55.524094
SB median: at min-max n_at 15 e 1440 fed_model val loss 2.4964 val acc 43.9765 best val_acc 51.976936 te_acc 55.524094
SB median: at min-max n_at 14 e 1450 fed_model val loss 2.3775 val acc 47.2740 best val_acc 51.976936 te_acc 55.524094
SB median: at min-max n_at 8 e 1460 fed_model val loss 2.4030 val acc 46.5172 best val_acc 51.976936 te_acc 55.524094
SB median: at min-max n_at 19 e 1470 fed_model val

In [21]:
def our_attack_score(all_updates, model_re, n_attackers, dev_type='unit_vec'):

    if dev_type == 'unit_vec':
        deviation = model_re / torch.norm(model_re)  # unit vector, dir opp to good dir
    elif dev_type == 'sign':
        deviation = torch.sign(model_re)
    elif dev_type == 'std':
        deviation = torch.std(all_updates, 0)
    
    lamda = torch.Tensor([50.0]).float().cuda()
    # print(lamda)
    threshold_diff = 1e-5
    lamda_fail = lamda
    lamda_succ = 0
    
    distances = []
    for update in all_updates:
        distance = torch.norm((all_updates - update), dim=1) ** 2
        distances = distance[None, :] if not len(distances) else torch.cat((distances, distance[None, :]), 0)
    
    scores = torch.sum(distances, dim=1)
    min_score = torch.min(scores)
    del distances

    while torch.abs(lamda_succ - lamda) > threshold_diff:
        mal_update = (model_re - lamda * deviation)
        distance = torch.norm((all_updates - mal_update), dim=1) ** 2
        score = torch.sum(distance)
        
        if score <= min_score:
            # print('successful lamda is ', lamda)
            lamda_succ = lamda
            lamda = lamda + lamda_fail / 2
        else:
            lamda = lamda - lamda_fail / 2

        lamda_fail = lamda_fail / 2

    # print(lamda_succ)
    mal_update = (model_re - lamda_succ * deviation)
    
    return mal_update
    

In [22]:
resume=0
nepochs=1500
gamma=.1
fed_lr=0.001

criterion = nn.CrossEntropyLoss()
use_cuda = torch.cuda.is_available()
batch_size = 100
schedule = [10000]
gamma=.5

aggregation='median'
candidates = []

at_type='min-sum'
dev_type='sign'
at_fractions=[20]

chkpt='./'+aggregation


for at_fraction in at_fractions:
    epoch_num = 0
    best_global_acc = 0
    best_global_te_acc = 0

    print('\n====> dev type %s batch size %d lr %f'% (dev_type, batch_size, fed_lr))    

    torch.cuda.empty_cache()

    fed_model = mnist_conv().cuda()
    fed_model.apply(weights_init)
    optimizer_fed = Adam(fed_model.parameters(), lr=fed_lr)

    while epoch_num <= nepochs:
        user_grads = []
        round_users = np.random.choice(3400, 60)
        n_attacker = np.sum(round_users < (34*at_fraction))

        for i in round_users:
            if i < (34*at_fraction):
                continue

            inputs = user_tr_data_tensors[i]
            targets = user_tr_label_tensors[i]

            inputs, targets = inputs.cuda(), targets.cuda()
            inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

            outputs = fed_model(inputs)
            loss = criterion(outputs, targets)
            optimizer_fed.zero_grad()
            loss.backward(retain_graph=True)

            param_grad=[]
            for param in fed_model.parameters():
                param_grad=param.grad.data.view(-1) if not len(param_grad) else torch.cat((param_grad,param.grad.view(-1)))

            user_grads=param_grad[None,:] if len(user_grads)==0 else torch.cat((user_grads,param_grad[None,:]),0)    

        malicious_grads = user_grads

        if epoch_num in schedule:
            for param_group in optimizer_fed.param_groups:
                param_group['lr'] *= gamma
                print('New learnin rate ', param_group['lr'])

        if n_attacker > 0:
            if at_type == 'lie':
                malicious_grads = get_malicious_updates_lie(malicious_grads, n_attacker, z, epoch_num)
            elif at_type == 'fang':
                agg_grads = torch.mean(malicious_grads, 0)
                deviation = torch.sign(agg_grads)
                malicious_grads = get_malicious_updates_fang(malicious_grads, agg_grads, deviation, n_attacker)
            elif at_type == 'our_attack_trmean':
                agg_grads = torch.mean(malicious_grads, 0)
                mal_update = our_attack_trmean(malicious_grads, agg_grads, n_attacker, dev_type)
            elif at_type == 'our_attack_median':
                agg_grads = torch.mean(malicious_grads, 0)
                mal_update = our_attack_median(malicious_grads, agg_grads, n_attacker, dev_type)
            elif at_type == 'min-max':
                agg_grads = torch.mean(malicious_grads, 0)
                mal_update = our_attack_dist(malicious_grads, agg_grads, n_attacker, dev_type)
            elif at_type == 'min-sum':
                agg_grads = torch.mean(malicious_grads, 0)
                mal_update = our_attack_score(malicious_grads, agg_grads, n_attacker, dev_type)

            mal_updates = torch.stack([mal_update] * n_attacker)
            malicious_grads = torch.cat((mal_updates, user_grads), 0)

        if epoch_num==0: print('malicious_grads shape ', malicious_grads.shape)

        if aggregation=='median':
            agg_grads=torch.median(malicious_grads,dim=0)[0]

        elif aggregation=='average':
            agg_grads=torch.mean(malicious_grads,dim=0)

        elif aggregation=='trmean':
            agg_grads=tr_mean(malicious_grads, n_attacker)

        del user_grads

        start_idx=0

        optimizer_fed.zero_grad()

        model_grads=[]

        for i, param in enumerate(fed_model.parameters()):
            param_=agg_grads[start_idx:start_idx+len(param.data.view(-1))].reshape(param.data.shape)
            start_idx=start_idx+len(param.data.view(-1))
            param_=param_.cuda()
            model_grads.append(param_)

        optimizer_fed.step(model_grads)

        val_loss, val_acc = test(val_data_tensor,val_label_tensor,fed_model,criterion,use_cuda)
        te_loss, te_acc = test(te_data_tensor,te_label_tensor, fed_model, criterion, use_cuda)

        is_best = best_global_acc < val_acc

        best_global_acc = max(best_global_acc, val_acc)

        if is_best:
            best_global_te_acc = te_acc

        if epoch_num%10==0 or epoch_num==nepochs-1:
            print('SB %s: at %s n_at %d e %d fed_model val loss %.4f val acc %.4f best val_acc %f te_acc %f'%(aggregation, at_type, n_attacker, epoch_num, val_loss, val_acc, best_global_acc,best_global_te_acc))

        epoch_num+=1


====> dev type sign batch size 100 lr 0.001000
malicious_grads shape  torch.Size([60, 848382])
SB median: at min-sum n_at 15 e 0 fed_model val loss 3.9859 val acc 5.4314 best val_acc 5.431425 te_acc 6.092978
SB median: at min-sum n_at 11 e 10 fed_model val loss 3.7809 val acc 4.8986 best val_acc 6.602657 te_acc 7.766166
SB median: at min-sum n_at 10 e 20 fed_model val loss 3.7622 val acc 8.0184 best val_acc 10.891166 te_acc 11.771520
SB median: at min-sum n_at 7 e 30 fed_model val loss 3.8071 val acc 23.7361 best val_acc 23.736100 te_acc 26.637150
SB median: at min-sum n_at 16 e 40 fed_model val loss 3.6608 val acc 30.0505 best val_acc 32.184411 te_acc 36.393122
SB median: at min-sum n_at 16 e 50 fed_model val loss 3.4543 val acc 32.2076 best val_acc 33.970861 te_acc 38.470449
SB median: at min-sum n_at 17 e 60 fed_model val loss 3.0690 val acc 35.4458 best val_acc 35.445840 te_acc 39.881075
SB median: at min-sum n_at 12 e 70 fed_model val loss 2.8766 val acc 34.6710 best val_acc 35.4

SB median: at min-sum n_at 14 e 690 fed_model val loss 1.8214 val acc 54.5459 best val_acc 58.507516 te_acc 60.407228
SB median: at min-sum n_at 11 e 700 fed_model val loss 1.7939 val acc 55.0196 best val_acc 58.507516 te_acc 60.407228
SB median: at min-sum n_at 13 e 710 fed_model val loss 1.7051 val acc 56.8884 best val_acc 58.507516 te_acc 60.407228
SB median: at min-sum n_at 9 e 720 fed_model val loss 1.7584 val acc 56.6052 best val_acc 58.507516 te_acc 60.407228
SB median: at min-sum n_at 10 e 730 fed_model val loss 1.7465 val acc 56.9811 best val_acc 58.507516 te_acc 60.407228
SB median: at min-sum n_at 16 e 740 fed_model val loss 1.7998 val acc 54.9887 best val_acc 58.507516 te_acc 60.407228
SB median: at min-sum n_at 9 e 750 fed_model val loss 1.7749 val acc 55.3104 best val_acc 58.507516 te_acc 60.407228
SB median: at min-sum n_at 8 e 760 fed_model val loss 1.7433 val acc 54.9320 best val_acc 58.625927 te_acc 60.878295
SB median: at min-sum n_at 10 e 770 fed_model val loss 1.71

SB median: at min-sum n_at 10 e 1390 fed_model val loss 1.7150 val acc 55.2075 best val_acc 60.826812 te_acc 62.646726
SB median: at min-sum n_at 13 e 1400 fed_model val loss 1.7510 val acc 55.2641 best val_acc 60.826812 te_acc 62.646726
SB median: at min-sum n_at 11 e 1410 fed_model val loss 1.6738 val acc 57.0248 best val_acc 60.826812 te_acc 62.646726
SB median: at min-sum n_at 11 e 1420 fed_model val loss 1.7309 val acc 55.5936 best val_acc 60.826812 te_acc 62.646726
SB median: at min-sum n_at 11 e 1430 fed_model val loss 1.6632 val acc 57.5422 best val_acc 60.826812 te_acc 62.646726
SB median: at min-sum n_at 14 e 1440 fed_model val loss 1.6388 val acc 57.6761 best val_acc 60.826812 te_acc 62.646726
SB median: at min-sum n_at 12 e 1450 fed_model val loss 1.6851 val acc 56.1548 best val_acc 60.826812 te_acc 62.646726
SB median: at min-sum n_at 11 e 1460 fed_model val loss 1.6602 val acc 57.1715 best val_acc 60.826812 te_acc 62.646726
SB median: at min-sum n_at 9 e 1470 fed_model va

In [12]:
resume=0
nepochs=1500
gamma=.1
fed_lr=0.001

criterion = nn.CrossEntropyLoss()
use_cuda = torch.cuda.is_available()
batch_size = 100
schedule = [5000]

aggregation = 'median'
arch = 'conv'
chkpt = './' + aggregation

at_type='fang'
at_fractions = [20, 5, 10, 15, 25]

for at_fraction in at_fractions:
    epoch_num = 0
    torch.cuda.empty_cache()
    torch.cuda.manual_seed_all(0)
    torch.cuda.manual_seed(0)
    np.random.seed(0)

    fed_file='fed_chkpt_%s_af_%d.pth.tar' % (at_type, at_fraction)
    fed_best_file='fed_best_%s_af_%d.pth.tar' % (at_type, at_fraction)

    fed_model = mnist_conv().cuda()
    fed_model.apply(weights_init)
    optimizer_fed = Adam(fed_model.parameters(), lr=fed_lr)

    best_global_acc=0
    best_global_te_acc=0

    if resume:
        fed_checkpoint = chkpt+'/'+fed_file
        assert os.path.isfile(fed_checkpoint), 'Error: no user checkpoint at %s'%(fed_checkpoint)
        checkpoint = torch.load(fed_checkpoint, map_location='cuda:%d'%torch.cuda.current_device())
        fed_model.load_state_dict(checkpoint['state_dict'])
        optimizer_fed.load_state_dict(checkpoint['optimizer'])
        resume = 0
        best_global_acc=checkpoint['best_acc']
        best_global_te_acc=checkpoint['best_te_acc']
        val_loss, val_acc = test(val_data_tensor,val_label_tensor,fed_model,criterion,use_cuda)
        epoch_num += checkpoint['epoch']
        print('resuming from epoch %d | val acc %.4f | best acc %.3f | best te acc %.3f'%(epoch_num, val_acc, best_global_acc, best_global_te_acc))

    candidates = []

    while epoch_num <= nepochs:
        user_grads = []

        # select 100 users randomly
        round_users = np.random.choice(3400,60)
        n_attackers = np.sum(round_users < (34*at_fraction))

        for i in round_users:
            if i < (34*at_fraction):
                continue

            inputs = user_tr_data_tensors[i]
            targets = user_tr_label_tensors[i]

            inputs, targets = inputs.cuda(), targets.cuda()
            inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

            outputs = fed_model(inputs)
            loss = criterion(outputs, targets)
            optimizer_fed.zero_grad()
            loss.backward(retain_graph=True)

            param_grad=[]
            for param in fed_model.parameters():
                param_grad=param.grad.data.view(-1) if not len(param_grad) else torch.cat((param_grad,param.grad.view(-1)))

            user_grads=param_grad[None,:] if len(user_grads)==0 else torch.cat((user_grads,param_grad[None,:]),0)    

        malicious_grads = user_grads

        if n_attackers:
            agg_grads = torch.mean(malicious_grads, 0)
            deviation = torch.sign(agg_grads)
            malicious_grads = get_malicious_updates_fang_trmean(malicious_grads, deviation, n_attackers, epoch_num)

        if aggregation=='median':
            agg_grads=torch.median(malicious_grads,dim=0)[0]
        elif aggregation=='average':
            agg_grads=torch.mean(malicious_grads,dim=0)
        elif aggregation=='trmean':
            agg_grads=tr_mean(malicious_grads, n_attackers)

        start_idx=0

        if epoch_num in schedule:
            for param_group in optimizer_fed.param_groups:
                param_group['lr'] *= gamma
                print('New learnin rate ', param_group['lr'])

        optimizer_fed.zero_grad()

        model_grads=[]

        for i, param in enumerate(fed_model.parameters()):
            param_=agg_grads[start_idx:start_idx+len(param.data.view(-1))].reshape(param.data.shape)
            start_idx=start_idx+len(param.data.view(-1))
            param_=param_.cuda()
            model_grads.append(param_)

        optimizer_fed.step(model_grads)

        val_loss, val_acc = test(val_data_tensor,val_label_tensor,fed_model,criterion,use_cuda)
        te_loss, te_acc = test(te_data_tensor,te_label_tensor, fed_model, criterion, use_cuda)

        is_best = best_global_acc < val_acc

        best_global_acc = max(best_global_acc, val_acc)

        if is_best:
            best_global_te_acc = te_acc

        save_checkpoint_global(
            {
                'epoch': epoch_num,
                'state_dict': fed_model.state_dict(),
                'optimizer': optimizer_fed.state_dict(),
                'best_acc': best_global_acc,
                'best_te_acc': best_global_te_acc,
            },
            is_best,
            checkpoint=chkpt,
            filename=fed_file,
            best_filename=fed_best_file,
        )

        if epoch_num % 10 == 0 or epoch_num == nepochs-1:
            print('%s: at %s n_at %d e %d fed_model val loss %.4f val acc %.4f best val_acc %f te_acc %f'%(aggregation, at_type, n_attackers, epoch_num, val_loss, val_acc, best_global_acc,best_global_te_acc))

        epoch_num+=1

median: at fang n_at 8 e 0 fed_model val loss 3.9401 val acc 6.4328 best val_acc 6.432764 te_acc 6.800865
median: at fang n_at 11 e 10 fed_model val loss 3.8243 val acc 5.3336 best val_acc 11.385400 te_acc 12.919584
median: at fang n_at 10 e 20 fed_model val loss 3.7888 val acc 10.3300 best val_acc 13.704695 te_acc 14.976318
median: at fang n_at 14 e 30 fed_model val loss 3.7258 val acc 9.1794 best val_acc 19.269975 te_acc 21.102759
median: at fang n_at 10 e 40 fed_model val loss 3.6957 val acc 17.0073 best val_acc 25.586903 te_acc 28.570325
median: at fang n_at 7 e 50 fed_model val loss 3.6282 val acc 27.6488 best val_acc 27.828974 te_acc 31.010605
median: at fang n_at 16 e 60 fed_model val loss 3.3522 val acc 27.2575 best val_acc 28.096685 te_acc 31.165054
median: at fang n_at 11 e 70 fed_model val loss 3.1304 val acc 29.1418 best val_acc 30.024712 te_acc 33.679984
median: at fang n_at 10 e 80 fed_model val loss 2.9737 val acc 32.3955 best val_acc 32.395490 te_acc 36.282434
median: a

median: at fang n_at 9 e 740 fed_model val loss 0.9551 val acc 70.3588 best val_acc 72.299732 te_acc 73.344831
median: at fang n_at 18 e 750 fed_model val loss 0.9680 val acc 70.4464 best val_acc 72.299732 te_acc 73.344831
median: at fang n_at 14 e 760 fed_model val loss 0.9460 val acc 70.9895 best val_acc 72.299732 te_acc 73.344831
median: at fang n_at 10 e 770 fed_model val loss 0.9164 val acc 71.8879 best val_acc 72.299732 te_acc 73.344831
median: at fang n_at 8 e 780 fed_model val loss 0.9565 val acc 70.2790 best val_acc 72.299732 te_acc 73.344831
median: at fang n_at 7 e 790 fed_model val loss 0.9362 val acc 71.3241 best val_acc 72.299732 te_acc 73.344831
median: at fang n_at 14 e 800 fed_model val loss 0.9877 val acc 70.3769 best val_acc 72.299732 te_acc 73.344831
median: at fang n_at 11 e 810 fed_model val loss 0.9594 val acc 71.0358 best val_acc 72.299732 te_acc 73.344831
median: at fang n_at 5 e 820 fed_model val loss 0.9209 val acc 70.7398 best val_acc 72.299732 te_acc 73.344

median: at fang n_at 10 e 1470 fed_model val loss 0.8688 val acc 73.7181 best val_acc 74.377059 te_acc 75.339786
median: at fang n_at 12 e 1480 fed_model val loss 0.8930 val acc 72.1221 best val_acc 74.377059 te_acc 75.339786
median: at fang n_at 14 e 1490 fed_model val loss 0.8781 val acc 72.1144 best val_acc 74.377059 te_acc 75.339786
median: at fang n_at 14 e 1499 fed_model val loss 0.9301 val acc 72.3358 best val_acc 74.377059 te_acc 75.339786
median: at fang n_at 16 e 1500 fed_model val loss 0.9602 val acc 72.0603 best val_acc 74.377059 te_acc 75.339786
median: at fang n_at 2 e 0 fed_model val loss 3.9578 val acc 4.7879 best val_acc 4.787891 te_acc 5.287273
median: at fang n_at 4 e 10 fed_model val loss 3.8065 val acc 5.3825 best val_acc 8.304160 te_acc 9.017195
median: at fang n_at 3 e 20 fed_model val loss 3.8027 val acc 5.3516 best val_acc 9.622117 te_acc 10.677512
median: at fang n_at 3 e 30 fed_model val loss 3.8068 val acc 15.8901 best val_acc 20.320222 te_acc 22.925247
medi

median: at fang n_at 4 e 690 fed_model val loss 1.1023 val acc 67.1103 best val_acc 68.003501 te_acc 69.457887
median: at fang n_at 3 e 700 fed_model val loss 1.1033 val acc 67.1180 best val_acc 68.003501 te_acc 69.457887
median: at fang n_at 3 e 710 fed_model val loss 1.0770 val acc 68.0782 best val_acc 68.078151 te_acc 69.550556
median: at fang n_at 2 e 720 fed_model val loss 1.0691 val acc 68.5235 best val_acc 68.523476 te_acc 69.756487
median: at fang n_at 6 e 730 fed_model val loss 1.1159 val acc 66.9764 best val_acc 68.523476 te_acc 69.756487
median: at fang n_at 2 e 740 fed_model val loss 1.1021 val acc 67.0974 best val_acc 68.523476 te_acc 69.756487
median: at fang n_at 4 e 750 fed_model val loss 1.1113 val acc 67.1412 best val_acc 69.254530 te_acc 70.469522
median: at fang n_at 5 e 760 fed_model val loss 1.0570 val acc 68.4540 best val_acc 69.254530 te_acc 70.469522
median: at fang n_at 3 e 770 fed_model val loss 1.0547 val acc 68.6676 best val_acc 69.254530 te_acc 70.469522
m

median: at fang n_at 1 e 1430 fed_model val loss 0.8502 val acc 73.2908 best val_acc 75.190486 te_acc 75.957578
median: at fang n_at 1 e 1440 fed_model val loss 0.8327 val acc 73.7052 best val_acc 75.190486 te_acc 75.957578
median: at fang n_at 1 e 1450 fed_model val loss 0.8424 val acc 74.0347 best val_acc 75.190486 te_acc 75.957578
median: at fang n_at 0 e 1460 fed_model val loss 0.8420 val acc 73.4993 best val_acc 75.190486 te_acc 75.957578
median: at fang n_at 2 e 1470 fed_model val loss 0.8198 val acc 74.3024 best val_acc 75.190486 te_acc 75.957578
median: at fang n_at 3 e 1480 fed_model val loss 0.8338 val acc 73.6975 best val_acc 75.190486 te_acc 75.957578
median: at fang n_at 3 e 1490 fed_model val loss 0.8388 val acc 73.9832 best val_acc 75.190486 te_acc 75.957578
median: at fang n_at 3 e 1499 fed_model val loss 0.8059 val acc 74.9871 best val_acc 75.190486 te_acc 75.957578
median: at fang n_at 3 e 1500 fed_model val loss 0.8188 val acc 74.2149 best val_acc 75.190486 te_acc 75

...

median: at fang n_at 7 e 1390 fed_model val loss 0.7531 val acc 76.0708 best val_acc 76.817339 te_acc 77.414539
median: at fang n_at 6 e 1400 fed_model val loss 0.7418 val acc 76.2536 best val_acc 76.817339 te_acc 77.414539
median: at fang n_at 4 e 1410 fed_model val loss 0.7605 val acc 75.9679 best val_acc 76.817339 te_acc 77.414539
median: at fang n_at 3 e 1420 fed_model val loss 0.7481 val acc 75.6204 best val_acc 76.817339 te_acc 77.414539
median: at fang n_at 5 e 1430 fed_model val loss 0.7475 val acc 75.7877 best val_acc 76.817339 te_acc 77.414539
median: at fang n_at 4 e 1440 fed_model val loss 0.7501 val acc 76.0271 best val_acc 76.817339 te_acc 77.414539
median: at fang n_at 5 e 1450 fed_model val loss 0.7589 val acc 75.8649 best val_acc 76.817339 te_acc 77.414539
median: at fang n_at 7 e 1460 fed_model val loss 0.7424 val acc 76.3617 best val_acc 76.817339 te_acc 77.414539
median: at fang n_at 3 e 1470 fed_model val loss 0.7479 val acc 75.5509 best val_acc 76.817339 te_acc 77

median: at fang n_at 7 e 610 fed_model val loss 1.0473 val acc 68.6136 best val_acc 69.414127 te_acc 70.788715
median: at fang n_at 6 e 620 fed_model val loss 1.0453 val acc 68.6934 best val_acc 69.414127 te_acc 70.788715
median: at fang n_at 11 e 630 fed_model val loss 1.0540 val acc 68.6007 best val_acc 69.414127 te_acc 70.788715
median: at fang n_at 11 e 640 fed_model val loss 1.0142 val acc 69.6870 best val_acc 70.039642 te_acc 71.277801
median: at fang n_at 10 e 650 fed_model val loss 1.0319 val acc 69.2983 best val_acc 70.039642 te_acc 71.277801
median: at fang n_at 9 e 660 fed_model val loss 1.0328 val acc 68.9328 best val_acc 70.039642 te_acc 71.277801
median: at fang n_at 12 e 670 fed_model val loss 1.0064 val acc 69.5969 best val_acc 70.039642 te_acc 71.277801
median: at fang n_at 16 e 680 fed_model val loss 1.0405 val acc 68.9070 best val_acc 70.309926 te_acc 71.630457
median: at fang n_at 9 e 690 fed_model val loss 1.0225 val acc 69.4270 best val_acc 70.309926 te_acc 71.630

median: at fang n_at 6 e 1350 fed_model val loss 0.8664 val acc 72.8866 best val_acc 74.073311 te_acc 74.938221
median: at fang n_at 12 e 1360 fed_model val loss 0.8286 val acc 73.6717 best val_acc 74.073311 te_acc 74.938221
median: at fang n_at 10 e 1370 fed_model val loss 0.8528 val acc 73.1724 best val_acc 74.436264 te_acc 74.984555
median: at fang n_at 13 e 1380 fed_model val loss 0.8340 val acc 73.7335 best val_acc 74.436264 te_acc 74.984555
median: at fang n_at 7 e 1390 fed_model val loss 0.8413 val acc 73.3320 best val_acc 74.436264 te_acc 74.984555
median: at fang n_at 4 e 1400 fed_model val loss 0.8399 val acc 73.4246 best val_acc 74.436264 te_acc 74.984555
median: at fang n_at 11 e 1410 fed_model val loss 0.8215 val acc 73.5971 best val_acc 74.436264 te_acc 74.984555
median: at fang n_at 12 e 1420 fed_model val loss 0.8200 val acc 73.8133 best val_acc 74.436264 te_acc 74.984555
median: at fang n_at 13 e 1430 fed_model val loss 0.8009 val acc 74.5418 best val_acc 74.541804 te_

median: at fang n_at 15 e 570 fed_model val loss 1.1665 val acc 66.1398 best val_acc 66.755045 te_acc 68.165671
median: at fang n_at 14 e 580 fed_model val loss 1.1697 val acc 65.3624 best val_acc 66.755045 te_acc 68.165671
median: at fang n_at 11 e 590 fed_model val loss 1.1603 val acc 65.4216 best val_acc 66.755045 te_acc 68.165671
median: at fang n_at 14 e 600 fed_model val loss 1.2009 val acc 65.9004 best val_acc 67.197797 te_acc 68.222302
median: at fang n_at 12 e 610 fed_model val loss 1.1840 val acc 64.9274 best val_acc 67.197797 te_acc 68.222302
median: at fang n_at 15 e 620 fed_model val loss 1.1603 val acc 65.6739 best val_acc 67.197797 te_acc 68.222302
median: at fang n_at 12 e 630 fed_model val loss 1.1909 val acc 65.2826 best val_acc 67.205519 te_acc 68.147652
median: at fang n_at 13 e 640 fed_model val loss 1.1878 val acc 64.5413 best val_acc 67.205519 te_acc 68.147652
median: at fang n_at 16 e 650 fed_model val loss 1.1848 val acc 65.7743 best val_acc 67.205519 te_acc 68

median: at fang n_at 10 e 1300 fed_model val loss 0.9893 val acc 70.1555 best val_acc 71.512047 te_acc 72.528830
median: at fang n_at 16 e 1310 fed_model val loss 0.9982 val acc 69.8157 best val_acc 71.512047 te_acc 72.528830
median: at fang n_at 15 e 1320 fed_model val loss 0.9856 val acc 69.9238 best val_acc 71.512047 te_acc 72.528830
median: at fang n_at 18 e 1330 fed_model val loss 1.0358 val acc 69.8595 best val_acc 71.512047 te_acc 72.528830
median: at fang n_at 17 e 1340 fed_model val loss 0.9943 val acc 69.4218 best val_acc 71.512047 te_acc 72.528830
median: at fang n_at 15 e 1350 fed_model val loss 1.0031 val acc 70.2070 best val_acc 71.512047 te_acc 72.528830
median: at fang n_at 12 e 1360 fed_model val loss 1.0208 val acc 69.0332 best val_acc 71.512047 te_acc 72.528830
median: at fang n_at 13 e 1370 fed_model val loss 1.0010 val acc 69.4630 best val_acc 71.512047 te_acc 72.528830
median: at fang n_at 16 e 1380 fed_model val loss 1.0752 val acc 68.1168 best val_acc 71.512047 